# Prototyping functions in nba_api 

query by player name and season

get points, assists, steals, blocks ,turnovers, rebounds

In [30]:
from nba_api.stats.static import players
from nba_api.stats.endpoints import commonplayerinfo, playergamelog
from nba_api.stats.static import players
import json
import pandas as pd

Make player ID -> photo association

In [75]:
bbr_to_name = json.load(open('player_bbr_to_name.json', 'r'))
bbr_to_pic = json.load(open('player_bbr_to_picture.json', 'r'))
bbr_to_name = bbr_to_name["results"]['bindings']

In [86]:
bbr_name_dict = {'name': [], 'bbr': []}

for entry in bbr_to_name:
    bbr_name_dict['name'].append(entry['name']['value'])
    bbr_name_dict['bbr'].append(entry['bbr']['value'])

data_df = pd.DataFrame.from_dict(bbr_name_dict)

In [87]:
# ~18 sec runtime
ids = []
urls = []

for row in data_df.iterrows():
    name_result = players.find_players_by_full_name(row[1]['name'])
    if name_result:
        player_id = name_result[0]['id']
        ids.append(int(player_id))
        try:
            urls.append(bbr_to_pic[row[1]['bbr']])
        except KeyError:
            urls.append(None)
    else:
        ids.append(None)
        urls.append(None)

data_df['player_id'] = ids
data_df['img_url'] = urls
data_df = data_df.drop(['name', 'bbr'], axis=1)
data_df = data_df.dropna()
data_df = data_df.astype({'player_id': 'int32'})


In [121]:
data_dict = data_df.set_index('player_id')['img_url'].to_dict()
json.dump(data_dict, open('player_id_to_img.json', 'w'))

In [7]:

# get_players returns a list of dictionaries, each representing a player.
nba_players = players.get_players()
#with open("all_players.json", "w+") as f:
#    jsonString = json.dump(nba_players, f)


{'id': 1630173,
 'full_name': 'Precious Achiuwa',
 'first_name': 'Precious',
 'last_name': 'Achiuwa',
 'is_active': True}

In [ ]:

import pandas as pd
name = 'lebron james'
season= '2018'
# remove 0 if want list of players
name_result = players.find_players_by_full_name(name)

player_found = True

if  len(name_result) != 1:
    player_found == False

player_to_query = ['id', 'full_name']
player_query = { rel_key: name_result[0][rel_key] for rel_key in player_to_query }

season_query = playergamelog.PlayerGameLog(player_id=player_query['id'], season = season)

stats_to_query = ['PTS', 'REB', 'AST','STL','BLK']
relevant_stats = season_query.get_data_frames()[0][stats_to_query]

stats_sum = relevant_stats.sum()
stats_avg = relevant_stats.mean()

for info in player_to_query:
    stats_sum[info] = player_query[info]
    stats_avg[info] = player_query[info]
    
print(stats_sum.to_json())
print(stats_avg.to_json())